In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option("display.max_columns", None)
sns.set(style="whitegrid")

In [ ]:
df_base = pd.read_csv("optimized_allocation_results.csv")
df_simulacoes = pd.read_csv("simulation_results_120cenarios.csv")

In [ ]:
colunas_essenciais = ["id", "predicted_proba", "risk_category", "simulated_cost", "true_label"]
verificacao = all(col in df_base.columns for col in colunas_essenciais)

print("df_base columns:", list(df_base.columns))
print("Valid structure?", verificacao)

df_base.head(10)

In [ ]:
def triagem_tradicional(df, metodo, capacidade_max, orcamento_total, min_alto_risco=0, random_state=42):
    """
    Performs traditional screening based on fixed criteria.
    Return: df with column 'selected' and metrics dictionary.
    """
    df_triagem = df.copy()
    np.random.seed(random_state)

    if metodo == "randomico":
        df_triagem = df_triagem.sample(frac=1).reset_index(drop=True)

    elif metodo == "apenas_alto_risco":
        df_triagem = df_triagem[df_triagem["risk_category"] == "alto"].copy()

    elif metodo == "por_proba_ordenada":
        df_triagem = df_triagem.sort_values(by="predicted_proba", ascending=False).copy()

    else:
        raise ValueError("❌ Unknown screening method.")

    selecionados = []
    custo_acumulado = 0

    for _, row in df_triagem.iterrows():
        if len(selecionados) >= capacidade_max:
            break
        if custo_acumulado + row["simulated_cost"] > orcamento_total:
            continue
        selecionados.append(row["id"])
        custo_acumulado += row["simulated_cost"]

    df["selecionado_tradicional"] = df["id"].apply(lambda x: 1 if x in selecionados else 0)

    df_sel = df[df["selecionado_tradicional"] == 1]
    risco_medio = df_sel["predicted_proba"].mean()
    positivos = df_sel[df_sel["true_label"] == 1].shape[0]
    altos = df_sel[df_sel["risk_category"] == "alto"].shape[0]

    return df, {
        "metodo": metodo,
        "atendidos": len(df_sel),
        "risco_medio": risco_medio,
        "custo_total": custo_acumulado,
        "positivos": positivos,
        "alto_risco": altos
    }


In [ ]:
params_teste = {
    "capacidade_max": 30,
    "orcamento_total": 18000,
    "min_alto_risco": 10
}

df_trad, metrics_trad = triagem_tradicional(df_base, metodo="por_proba_ordenada", **params_teste)

print("Traditional screening metrics (por_proba_ordenada):")
print(metrics_trad)
df_trad[df_trad["selecionado_tradicional"] == 1].head(10)

In [ ]:
metodos = ["randomico", "apenas_alto_risco", "por_proba_ordenada"]
resultados_trad = []

for metodo in metodos:
    _, metricas = triagem_tradicional(df_base, metodo=metodo, **params_teste)
    resultados_trad.append(metricas)

df_tradicional = pd.DataFrame(resultados_trad)
df_tradicional

In [ ]:
plt.figure(figsize=(10, 6))
sns.barplot(data=df_tradicional, x="metodo", y="risco_medio", palette="Set2")
plt.title("Comparison of Average Risk by Screening Strategy")
plt.ylabel("Average Risk of Selected Patients")
plt.xlabel("Screening Method")
plt.tight_layout()
plt.show()

In [ ]:
cenario_otimizado = df_simulacoes.query(
    "capacidade_max == 30 and orcamento_total == 18000 and min_alto_risco == 10 and peso_custo == 0.001"
)

if not cenario_otimizado.empty:
    linha_otimizada = cenario_otimizado.iloc[0].to_dict()
    linha_otimizada.update({"metodo": "modelo_otimizado"})
    df_comparativo = pd.concat([df_tradicional, pd.DataFrame([linha_otimizada])], ignore_index=True)
    print("Optimized scenario added successfully.")
else:
    print("Optimized scenario with specified parameters not found.")
    df_comparativo = df_tradicional.copy()

df_comparativo

In [ ]:
df_comparativo.to_csv("comparacao_modelos_tradicionais_vs_otimizado.csv", index=False)